In [1]:
import sys
import dotenv
dotenv.load_dotenv('../../.env')

sys.path.insert(0, '../')
sys.path.insert(0, '../app')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:182: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from app.S3Storage import S3Storage


urls = [
    # "https://www.youtube.com/watch?v=re__12nH2ro",
    # "https://www.youtube.com/watch?v=FSbEorqTtMU",
    # "https://www.youtube.com/watch?v=Ull-Rz1L-J0",
    # "https://www.youtube.com/watch?v=0oB2oBj5Ex4",
    # "https://www.youtube.com/watch?v=p5JR8fsEzR4",
    "https://www.youtube.com/watch?v=tRWVMdkOCoQ",
]

s3 = S3Storage()
with SIWeaviateClient() as client:
    for url in urls:
        YoutubeProcessor(s3=s3,client=client, whisper=whisper, youtube_url=url)

AUDIO CODEC aac


whisper-medium: 100%|██████████| 723.139/723.139 [00:34<00:00, 21.26s/s]


time_taken 35.05392003059387
File /home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/documents/youtube/af1102c9-8cae-4e56-bd35-e5ecf23a8373.mp4 uploaded successfully to bucket science-infuse-content-dev as youtube/bc15c43d-ac78-4d13-84c5-74c8a9d3931a.mp4.


# tests

In [5]:
from pytubefix import YouTube
url =     "https://www.youtube.com/watch?v=Ull-Rz1L-J0"
yt = YouTube(url)
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
# yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()#.download(output_path=output_path, filename=filename)


In [6]:
yt.streams

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="25fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="25fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">, <Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">]

In [35]:
from googleapiclient.discovery import build

# Set up YouTube API client
api_key = ''
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    playlist_id = f"{channel_id[0]}U{channel_id[2:]}"
    videos = []
    next_page_token = None
    
    while True:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        
        for item in res["items"]:
            videos.append({
                "title": item["snippet"]["title"],
                "video_id": item["snippet"]["resourceId"]["videoId"],
                "publish_time": item["snippet"]["publishedAt"]
            })
        
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
            
    return videos

videos = get_channel_videos("UC1udnO-W6gpR9qzleJ5SDKw")
print(len(videos))

442


In [25]:
videos